In [3]:
# 文件保存在google drive
# 在colab调用前需要有认证
from google.colab import drive
drive.mount('/content/drive/')
datapath = 'drive/MyDrive/现代优化计算方法/assign100.txt'

Mounted at /content/drive/


In [ ]:
datapath='assign100.txt'

# 文件读取


In [4]:
f=open(datapath)
data=f.readlines()  #逐行读取txt并存成list。每行是list的一个元素，数据类型为str
l=[]
# for i in range(len(data)):  #len(data)为数据行数
#     #data.seek(0,0)
#     for j in range(len(list(data[0].split()))):   #len(list(data[0].split()))为数据列数
#         l.append(data[i].split(' ')[j])

In [5]:
# 使用超参数来保证对于所有此类问题都有解
scale = int(data[0].split()[0])
scale

100

data[2].replace('\n','')
data[2].replace(r''','')

In [6]:
timetable = data[1::]
timetable[-1]

' 30 23 42 5 24 53 64 76 4\n'

### 转换成一长条

In [7]:
all_timetable=''
for i in timetable:
  all_timetable = all_timetable + i
all_timetable = all_timetable.split()

this = []
for i in all_timetable:
  this.append(int(i))
all_timetable = this

list_timetable = []
for i in range(len(timetable)):
  thisline = timetable[i].split()
  tmp = []
  for j in range(len(thisline)):
    tmp.append( int(timetable[i].split()[j]) )
  list_timetable.append(tmp)
list_timetable

In [8]:
len(all_timetable)

10000

### 从一长条转换成scale宽度的矩阵
记得检查  
还可以用numpy的reshape

In [9]:
import numpy as np
import random
import matplotlib.pyplot as plt

In [10]:
matrix = []
for i in range(int(len(all_timetable)/scale)):
  tmp = all_timetable[i:i+scale]
  matrix.append(tmp)
# 检查最后一行有没有满
len(matrix[-1]),type(matrix[1][2])

(100, int)

In [11]:
# 数组长度不匹配，转换的array 不是二维的，只能是list。
# 方法一：分开转换list
# 方法二：直接对齐

In [12]:
time = np.array(matrix)
# time.dtype = 'int32'
time.shape

(100, 100)

time


time = np.asarray([list_timetable])
time

# 算法

In [20]:
from tqdm import tqdm, trange

In [16]:
def get_rand():
    select = [x for x in range(100)]
    random.shuffle(select)
    return select

In [ ]:
def cal_time(rang, time, pop):
    sum_time_pop = []
    for choice in rang:
        single_array_sum_time = 0
        for i in range(pop):
            cut = choice[i]
            single_array_sum_time += time[cut][i]
        sum_time_pop.append(single_array_sum_time)
    sum_time_pop = np.array(sum_time_pop).reshape(-1, 1)
    num = sum_time_pop.shape[0]
    return sum_time_pop, num

In [ ]:
# 定义一个修饰器函数用来统计函数的运行时间
# 参考我的csdn  https://blog.csdn.net/prinTao/article/details/121800857?spm=1001.2014.3001.5501
import time
def timmer(func):    #传入的参数是一个函数
    def deco(*args, **kwargs): #本应传入运行函数的各种参数
        print('\n函数：{_funcname_}开始运行：'.format(_funcname_=func.__name__))
        start_time = time.time()#调用代运行的函数，并将各种原本的参数传入
        res = func(*args, **kwargs)
        end_time = time.time()
        print('函数:{_funcname_}运行了 {_time_}秒'
              .format(_funcname_=func.__name__, _time_=(end_time - start_time)))
        return res#返回值为函数
    return deco

In [ ]:
bestpop = []  # 最优种群
itertimes = 1000
pm = 0.1
pc = 0.8
# 种群数量
magnit = 1000
# 个体数量
pop = (time.shape)[1]
# 初始化种群
rang = np.zeros((magnit, pop), int)
for i in range(magnit):
    select = [x for x in range(scale)]
    random.shuffle(select)
    rang[i, :] = select

for count in tqdm(range(itertimes), desc='Searching'):

    # 计算每一种分配的总value
    sum_time_pop = []
    for choice in rang:
        single_array_sum_time = 0
        for i in range(pop):
            cut = choice[i]
            single_array_sum_time += time[cut][i]
        sum_time_pop.append(single_array_sum_time)
    sum_time_pop = np.array(sum_time_pop).reshape(-1, 1)
    num = sum_time_pop.shape[0]

    # 排序
    index = np.argsort(sum_time_pop, 0)
    sum_time_pop = sum_time_pop[index].reshape(num, 1)[0:magnit, :]
    rang = rang[index].reshape(num, pop)[0:magnit, :]

    # 计算适值和适应度
    fit = 1000 - sum_time_pop[:]        # 适值
    fitplus = np.cumsum(fit).reshape(-1, 1)  # 适值向下叠加
    # print(fitplus[magnit-1, :])
    total_fit_value = fitplus[:] / fitplus[magnit - 1, :]  # 适应度向下叠加
    total_fit_value = np.insert(total_fit_value, 0, np.array([0]), 0)  # 在第一行添加0

    # 选择
    new_solution = []
    new_sum_time = []
    for _ in range(magnit):
        rand = random.random()
        for individual in range(magnit):
            if rand > total_fit_value[individual, :] and rand < total_fit_value[individual + 1, :]:
                new_solution.append(rang[individual, :])
                new_sum_time.append(sum_time_pop[individual, :])
                continue
    new_solution = np.array(new_solution)
    prechoose = new_solution.copy()
    new_sum_time = np.array(new_sum_time)
    # 交叉
    for individual in range(0, (np.shape(new_solution))[0], 2):
        if pc > random.random():
            chrome_a = new_solution[individual, :].copy()
            chrome_b = new_solution[individual + 1, :].copy()
            select = [x for x in range(10)]
            random.shuffle(select)
            this_A = select[0]
            this_B = select[1]
            this_A, this_B = min(this_A, this_B), max(this_A, this_B)
            cthis_A, cthis_B = chrome_a[this_A:this_B + 1].copy(), chrome_b[this_A:this_B + 1].copy()
            # print(this_A, this_B, cthis_A, cthis_B)
            for site in range(this_B - this_A + 1):
                s1 = np.where(chrome_a == cthis_B[site])
                chrome_a[s1] = (chrome_a[this_A:this_B + 1])[site].copy()
                a = chrome_a
            for site in range(this_B - this_A + 1):
                s2 = np.where(chrome_b == cthis_A[site])
                chrome_b[s2] = (chrome_b[this_A:this_B + 1])[site].copy()
                b = chrome_b
            chrome_a[this_A:this_B + 1] = cthis_B.copy()
            chrome_b[this_A:this_B + 1] = cthis_A.copy()
            new_solution[individual, :] = chrome_a.copy()
            new_solution[individual + 1, :] = chrome_b.copy()
    # 变异
    for individual in range(0, (np.shape(new_solution))[0]):
        if pm > random.random():
            select = [x for x in range(10)]
            random.shuffle(select)
            this_A = select[0]
            this_B = select[1]
            gen = new_solution[individual, :]
            gen[this_A], gen[this_B] = gen[this_B], gen[this_A]
            new_solution[individual, :] = gen.copy()
    rang = np.append(prechoose, new_solution, 0)
    bestpop.append(sum_time_pop[0, :])
    
bestpop = np.array(bestpop)
print(sum_time_pop[0, :])
print(rang[0, :])

x = [a for a in range(1,1+itertimes)]
y = bestpop
plt.plot(x,y)
plt.xlabel('iter')
plt.ylabel('time')
plt.show()


Searching:  18%|█▊        | 175/1000 [06:04<27:27,  2.00s/it]